**IMPORTING LIBRARIES**

In [7]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer
from sklearn import svm
from textblob import TextBlob
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import classification_report
from sklearn import tree
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import GridSearchCV


In [8]:
import random

def sentiment_func(label):
    if label <0:
        return "NEGATIVE"
    elif label <= 0.2:
        return "NEUTRAL"
    elif label > 0.2:
        return "POSITIVE"
        

In [9]:
df = pd.read_csv(r'Datafiniti_Hotel_Reviews_Jun19.csv')
df

,reviews.text
0,"This hotel was nice and quiet. Did not know, t..."
1,We stayed in the king suite with the separatio...
2,"Parking was horrible, somebody ran into my ren..."
3,Not cheap but excellent location. Price is som...
4,If you get the room that they advertised on th...
...,...
9995,My friends and I took a trip to Hampton for th...
9996,"from check in to departure, staff is friendly,..."
9997,This Hampton is located on a quiet street acro...
9998,Awesome wings (my favorite was garlic parmesan...


In [10]:
df = df.loc[:, df.columns.intersection(['reviews.text'])]
inplace=True
df

,reviews.text
0,"This hotel was nice and quiet. Did not know, t..."
1,We stayed in the king suite with the separatio...
2,"Parking was horrible, somebody ran into my ren..."
3,Not cheap but excellent location. Price is som...
4,If you get the room that they advertised on th...
...,...
9995,My friends and I took a trip to Hampton for th...
9996,"from check in to departure, staff is friendly,..."
9997,This Hampton is located on a quiet street acro...
9998,Awesome wings (my favorite was garlic parmesan...


In [11]:
df.info()
df.describe()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 1 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   reviews.text  10000 non-null  object
dtypes: object(1)
memory usage: 78.2+ KB


,reviews.text
count,10000
unique,9770
top,MoreMore
freq,18


In [12]:
def review_polarity(text):
    return TextBlob(text).sentiment.polarity

In [13]:
df['polarity'] = df['reviews.text'].apply(review_polarity)
df.head(10)

,reviews.text,polarity
0,"This hotel was nice and quiet. Did not know, t...",0.187500
1,We stayed in the king suite with the separatio...,0.399524
2,"Parking was horrible, somebody ran into my ren...",-0.350000
3,Not cheap but excellent location. Price is som...,0.495238
4,If you get the room that they advertised on th...,0.214286
5,"This is such a fun, lovely hotel. The attentio...",0.198077
6,We recently stayed at this hotel on a trip to ...,0.182143
7,"I reserved a room a week in advance, knowing a...",-0.400000
8,MoreMore,0.000000
9,"Hello, I have traveled a lot and abroad and by...",-0.342857


In [14]:
df['sentiment'] = df['polarity'].apply(sentiment_func)
df.head(18)

,reviews.text,polarity,sentiment
0,"This hotel was nice and quiet. Did not know, t...",0.187500,NEUTRAL
1,We stayed in the king suite with the separatio...,0.399524,POSITIVE
2,"Parking was horrible, somebody ran into my ren...",-0.350000,NEGATIVE
3,Not cheap but excellent location. Price is som...,0.495238,POSITIVE
4,If you get the room that they advertised on th...,0.214286,POSITIVE
5,"This is such a fun, lovely hotel. The attentio...",0.198077,NEUTRAL
6,We recently stayed at this hotel on a trip to ...,0.182143,NEUTRAL
7,"I reserved a room a week in advance, knowing a...",-0.400000,NEGATIVE
8,MoreMore,0.000000,NEUTRAL
9,"Hello, I have traveled a lot and abroad and by...",-0.342857,NEGATIVE


In [15]:
rating = df.iloc[:,2]
review = df.iloc[:,0]

In [16]:
review_train, review_test, rating_train, rating_test = train_test_split(review, rating, test_size=0.2, random_state=42)

In [17]:
print(rating_train)

9254     NEUTRAL
1561    POSITIVE
1670    POSITIVE
6087     NEUTRAL
6669    POSITIVE
          ...   
5734    POSITIVE
5191    POSITIVE
5390     NEUTRAL
860     POSITIVE
7270    POSITIVE
Name: sentiment, Length: 8000, dtype: object


In [18]:
count_vect = CountVectorizer()
# rvwtrain=['review_train']
# rvwtest=['review_test']
# ratetrain=['rating_train']
# ratetest=['rating_test']

X_train_counts = count_vect.fit_transform(review_train)
X_test_counts = count_vect.transform(review_test)
X_train_counts.shape

(8000, 19938)

In [19]:
# print(X_train_counts[0].toarray())
Index_label = df[df['polarity']==0.2].index.tolist()
  
# Print all the labels
print(Index_label)

[446, 689, 2029, 2130, 2229, 2657, 3460, 3466, 3498, 3801, 4077, 5177, 8505, 8661]


In [20]:
svm_class = svm.SVC(C=0.1, gamma=1, kernel='linear')
svm_class.fit(X_train_counts,rating_train)
# review_test[0]
svmpredict=svm_class.predict(X_test_counts)
print(classification_report(rating_test, svmpredict))

              precision    recall  f1-score   support

    NEGATIVE       0.57      0.49      0.53        98
     NEUTRAL       0.70      0.64      0.67       468
    POSITIVE       0.91      0.94      0.92      1434

    accuracy                           0.85      2000
   macro avg       0.72      0.69      0.71      2000
weighted avg       0.84      0.85      0.84      2000



In [21]:

knn = KNeighborsClassifier(n_neighbors=3)

knn.fit(X_train_counts, rating_train)
predictedknn = knn.predict(X_test_counts)
# Calculate the accuracy of the model
print(classification_report(rating_test, predictedknn))

              precision    recall  f1-score   support

    NEGATIVE       0.19      0.14      0.16        98
     NEUTRAL       0.45      0.24      0.31       468
    POSITIVE       0.77      0.90      0.83      1434

    accuracy                           0.71      2000
   macro avg       0.47      0.43      0.43      2000
weighted avg       0.67      0.71      0.68      2000



In [22]:
dt = tree.DecisionTreeClassifier()
dt = dt.fit(X_train_counts, rating_train)
predictdt = dt.predict(X_test_counts)
print(classification_report(rating_test, predictdt))
# max_depth = 5 , criterion = 'entropy', random_state = 42

              precision    recall  f1-score   support

    NEGATIVE       0.36      0.33      0.34        98
     NEUTRAL       0.46      0.43      0.44       468
    POSITIVE       0.83      0.86      0.85      1434

    accuracy                           0.73      2000
   macro avg       0.55      0.54      0.55      2000
weighted avg       0.72      0.73      0.73      2000



In [23]:
log_classifier = LogisticRegression(multi_class='multinomial', solver='lbfgs',random_state = 42)
log_classifier.fit(X_train_counts, rating_train)
logpred = log_classifier.predict(X_test_counts)
print(classification_report(rating_test, logpred))

              precision    recall  f1-score   support

    NEGATIVE       0.60      0.42      0.49        98
     NEUTRAL       0.69      0.65      0.67       468
    POSITIVE       0.90      0.94      0.92      1434

    accuracy                           0.85      2000
   macro avg       0.73      0.67      0.69      2000
weighted avg       0.84      0.85      0.84      2000



C:\Users\HP\anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:763: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


In [24]:
param_grid = {'C': [0.1, 1], 
              'gamma': [1, 0.1, 0.01, 0.001, 0.0001],
              'kernel': ['rbf','linear']} 
  
grid = GridSearchCV(svm.SVC(), param_grid, refit = True, verbose = 3)
  
# fitting the model for grid search
grid.fit(X_train_counts, rating_train)
print(grid.best_params_)
print(grid.best_estimator_)

Fitting 5 folds for each of 20 candidates, totalling 100 fits
[CV 1/5] END ........C=0.1, gamma=1, kernel=rbf;, score=0.725 total time=  58.3s
[CV 2/5] END ........C=0.1, gamma=1, kernel=rbf;, score=0.726 total time= 1.0min
[CV 3/5] END ........C=0.1, gamma=1, kernel=rbf;, score=0.726 total time= 1.0min
[CV 4/5] END ........C=0.1, gamma=1, kernel=rbf;, score=0.727 total time=  54.2s
[CV 5/5] END ........C=0.1, gamma=1, kernel=rbf;, score=0.724 total time=  54.1s
[CV 1/5] END .....C=0.1, gamma=1, kernel=linear;, score=0.848 total time=  17.5s
[CV 2/5] END .....C=0.1, gamma=1, kernel=linear;, score=0.861 total time=  20.1s
[CV 3/5] END .....C=0.1, gamma=1, kernel=linear;, score=0.851 total time=  17.2s
[CV 4/5] END .....C=0.1, gamma=1, kernel=linear;, score=0.831 total time=  17.0s
[CV 5/5] END .....C=0.1, gamma=1, kernel=linear;, score=0.849 total time=  17.4s
[CV 1/5] END ......C=0.1, gamma=0.1, kernel=rbf;, score=0.725 total time=  53.0s
[CV 2/5] END ......C=0.1, gamma=0.1, kernel=rbf